# Demonstration of the funds_py package
Project Link: github.com/joaopm33/funds_py <br>
Documentation: joaopm33.github.io/fundspy/docs/fundspy.html <br>
Author: Joao Penido Monteiro <br>
Linkedin: linkedin.com/in/joao-penido-monteiro/

## 1. Import the needed libraries

In [1]:
!pip install fundspy

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.2/227.2 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 16.6 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [3]:
from fundspy.fundspy import returns, cum_returns, volatility, drawdown, corr_benchmark, beta, alpha, sharpe, sortino, capture_ratio
import pandas as pd
import sqlite3
import plotly.express as px

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 500)
pd.options.display.float_format = '{:,.4f}'.format

ModuleNotFoundError: No module named 'fundspy'

## 2. Connect to the database

In [ ]:
#connects to the database
con = sqlite3.connect(r"investments_database.db")

## 3. Pull the data from the database to a pandas dataframe
The SQL query bellow can seem a bit complicated at a first glance, but the steps are actually pretty simple: <br>

&ensp;&ensp;1 - Defines a filter that will be used in step 2: we only keep to our dataset funds that had at least 200 clients and 50 million reais in net assets during the month of march 2021. <br>

&ensp;&ensp;2 - Defines our funds table: we select the funds CNPJ (unique identifier), date and daily quota prices. We also join the funds name from the cadastral information table. At this step, a new filter is added: only funds classified in the stocks category are kept. Finally, we keep in our result set only data starting from the year of 2009. <br>

&ensp;&ensp;3 - Defines the selic and ibovespa tables. <br>

&ensp;&ensp;4 - Joins the tables defined in steps 2 and 3, ordering it by date and fund CNPJ.

In [ ]:
query = '''
WITH
-- defines the filter table we will use in our query
filter_query AS
(
SELECT DISTINCT CNPJ_FUNDO
FROM daily_quotas
WHERE DT_COMPTC >= '2021-03-01' AND
      NR_COTST >= 200 AND
      VL_PATRIM_LIQ >= 50000000
),

--defines the table with funds daily quotas
funds AS
(SELECT A.*, B.fund_name FROM
(
SELECT DT_COMPTC as date, CNPJ_FUNDO as cnpj_fund, VL_QUOTA as fund_quota
FROM daily_quotas
WHERE CNPJ_FUNDO IN (select * from filter_query) AND
      DT_COMPTC >= '2009-01-01')A
INNER JOIN
(
 SELECT DISTINCT CNPJ_FUNDO, DENOM_SOCIAL as fund_name
 FROM info_cadastral_funds
 WHERE julianday('now') - julianday(DT_INI_ATIV) >= 730 AND
 CLASSE = 'Fundo de Ações'
)B
    ON A.cnpj_fund = B.CNPJ_FUNDO
),

--defines the table with the ibovespa prices
ibov AS
(
SELECT Date, Close AS ibov_price
FROM ibov_returns
),

-- defines the table with the selic (interest rates) returns
selic AS
(
SELECT date, price as selic_price
FROM selic_rates
)

SELECT funds.*, ibov.ibov_price, selic.selic_price from
funds
LEFT JOIN
ibov
    ON funds.date = ibov.Date
LEFT JOIN
selic
    ON funds.date = selic.date
ORDER BY date, cnpj_fund
'''

In [ ]:
funds = pd.read_sql(query, con)#executes the query and reads the data

#fills any null values we had in our dataset with the following day value
cols = ['fund_quota', 'ibov_price', 'selic_price']
funds[cols] = funds[cols].fillna(method = 'backfill')

#lets see how our dataset looks like
funds.head(5)

## 4. Analyze the data

### returns function
We have prices for each fund and ibovespa, but to actually do any comparation, we need percentual returns. So we'll use the returns function of the funds_py library to calculate them. <br>
Techinical reference: investopedia.com/terms/c/cagr.asp


In [ ]:
#Top 5 funds by CAGR
funds_cagr = returns(funds, group = 'cnpj_fund', values = ['fund_quota', 'ibov_price', 'selic_price'], rolling = False)
funds_cagr = funds_cagr.merge(funds.groupby('cnpj_fund')[['fund_name']].first(),
                              how = 'left',
                              left_index = True,
                              right_index = True
                             )

funds_cagr.sort_values('fund_quota_cagr', ascending = False).head()

To compare funds that had diferent start dates, using rolling windows can be useful. Use the returns function with a 1 year rolling windows of returns to plot the top 5 funds returns.

In [ ]:
top5 = (funds_cagr.sort_values('fund_quota_cagr', ascending = False)
         .reset_index()
         .iloc[:5,0] #selects top 5 rows and cnpj column
         .to_list()
        )

#calculates top 5 funds (cagr) 1 year rolling window performance
top5_1rw = returns(funds[funds.cnpj_fund.isin(top5)],
                   rolling = True,
                   window_size = 252,
                   group = 'cnpj_fund',
                   values = ['fund_quota', 'ibov_price', 'selic_price']
                   )

#plots the top 5 funds performance over time
fig = px.line(top5_1rw,
              x='date',
              y='fund_quota_return_252d',
              color = 'fund_name' ,
              title='Top 5 Funds(by CAGR) 1 Year Rolling Window Returns Over Time',
              labels={"fund_quota_return_252d": "Returns 1 year rolling window", "fund_name": "Fund name", "date":"Date"})


fig.update_layout(legend=dict(
    yanchor="bottom",
    y= -0.7,
    xanchor="left",
    x=0.01
))

fig.show()

### cum_returns function
You can see how the performance of the top 5 funds was build over time.
Techinical reference: investopedia.com/terms/c/cumulativereturn.asp

In [ ]:
top5_returns = cum_returns(funds[funds.cnpj_fund.isin(top5)],
                            group = 'cnpj_fund',
                            values = ['fund_quota', 'ibov_price', 'selic_price']
                           )

#plots the top 5 funds performance over time
fig = px.line(top5_returns,
              x='date',
              y='fund_quota_cum_return',
              color = 'fund_name' ,
              title='Top 5 Funds (by CAGR) Performance Over Time',
              labels={"fund_quota_cum_return": "Cumulative return", "fund_name": "Fund name", "date":"Date"})


fig.update_layout(legend=dict(
    yanchor="bottom",
    y= -0.7,
    xanchor="left",
    x=0.01
))

fig.show()

### drawdown function
Lets take a look at how underwater (% loss from all time high) each fund in the top 5 was over time. <br>
Techinical reference: investopedia.com/terms/d/drawdown.asp

In [ ]:
top5_drawdown = drawdown(funds[funds.cnpj_fund.isin(top5)],
                            group = 'cnpj_fund',
                            values = ['fund_quota', 'ibov_price']
                           )
#plots the top 5 funds drawdown over time
fig = px.line(top5_drawdown,
              x='date',
              y='drawdown_fund_quota',
              color = 'fund_name' ,
              title='Top 5 Funds (by CAGR) Drawdown Over Time',
              labels={"drawdown_fund_quota": "Drawdown", "fund_name": "Fund name", "date":"Date"})


fig.update_layout(legend=dict(
    yanchor="bottom",
    y= -0.7,
    xanchor="left",
    x=0.01
))

fig.show()

### volatility function
Calculates the annualized volatility of each fund <br>
Techinical reference: investopedia.com/terms/v/volatility.asp

#### Vollatility over the complete lifetime of the asset
One consolidated measure per asset

In [ ]:
##5 most volatile funds

#Calculates daily returns of each fund
funds_returns = returns(funds, group = 'cnpj_fund', values = ['fund_quota', 'ibov_price'], rolling = True)

#calculates the standard deviation of daily returns
funds_vol = volatility(funds_returns, group = 'cnpj_fund', values = ['fund_quota_return_1d', 'ibov_price_return_1d'], rolling = False)

funds_vol = funds_vol.merge(funds.groupby('cnpj_fund')[['fund_name']].first(),
                              how = 'left',
                              left_index = True,
                              right_index = True
                             )

funds_vol.sort_values('fund_quota_return_1d_vol', ascending = False).head()

#### Vollatility in rolling windows
Track changes and tendencies over time.

In [ ]:
#calculates the standard deviation of daily returns
funds_vol_mthRW = volatility(funds_returns,
                             group = 'cnpj_fund',
                             values = ['fund_quota_return_1d', 'ibov_price_return_1d'],
                             rolling = True, window_size = 21)

funds_vol_mthRW = funds_vol_mthRW[funds_vol_mthRW.cnpj_fund.isin(top5)]


#plots the top 5 funds drawdown over time
fig = px.line(funds_vol_mthRW,
              x='date',
              y='fund_quota_return_1d_vol_21rw',
              color = 'fund_name' ,
              title='Top 5 Funds (by CAGR) Annualized Volatility Over Time',
              labels={"fund_quota_return_1d_vol_21rw": "annualized volatility", "fund_name": "Fund name", "date":"date"})


fig.update_layout(legend=dict(
    yanchor="bottom",
    y= -0.7,
    xanchor="left",
    x=0.01
))

fig.show()

### corr_benchmark function
how correlated are our top 5 CAGR funds in comparisson to the ibovespa index? <br>
Techinical reference: investopedia.com/terms/c/correlationcoefficient.asp

#### Correlation over the complete lifetime of the asset
One consolidated measure per asset

In [ ]:
#calculates the correlation of daily returns with the benchmark
funds_corr = corr_benchmark(funds_returns,'fund_quota_return_1d', 'ibov_price_return_1d', group = 'cnpj_fund'
                            , rolling = False)

funds_corr = funds_corr.merge(funds.groupby('cnpj_fund')[['fund_name']].first(),
                              how = 'left',
                              left_index = True,
                              right_index = True
                             )
funds_corr.loc[top5, :].sort_values('correlation_benchmark', ascending = False).head()

#### Correlation in rolling windows
Track changes and tendencies over time.

In [ ]:
#calculates the correlation of daily returns with the benchmark in 1 month rolling windows
funds_corr_mthRW = corr_benchmark(funds_returns,
                                  group = 'cnpj_fund',
                                  asset_returns = 'fund_quota_return_1d',
                                  index_returns = 'ibov_price_return_1d',
                                  rolling = True,
                                  window_size = 252)

funds_corr_mthRW = funds_corr_mthRW[funds_corr_mthRW.cnpj_fund.isin(top5)].sort_values('date')


#plots the top 5 funds drawdown over time
fig = px.line(funds_corr_mthRW,
              x='date',
              y='correlation_benchmark',
              color = 'fund_name' ,
              title='Top 5 Funds (by CAGR) Correlation With Ibovespa Over Time',
              labels={"correlation_benchmark": "correlation with ibovespa", "fund_name": "Fund name", "date":"date"})


fig.update_layout(legend=dict(
    yanchor="bottom",
    y= -0.7,
    xanchor="left",
    x=0.01
))

fig.show()

### beta function
Having both the volatilities of the funds and their correlations with the ibovespa index, we can easily calculate their betas (measure of the volatility of an asset compared to the market) <br>
Techinical reference: investopedia.com/terms/b/beta.asp

In [ ]:
#joins correlation and volatilities dataframes
funds_beta = funds_vol.drop(columns = 'fund_name').join(funds_corr)

#calculates betas
funds_beta = beta(funds_beta,
                  asset_vol = 'fund_quota_return_1d_vol' ,
                  bench_vol = 'ibov_price_return_1d_vol',
                  correlation = 'correlation_benchmark')

#Betas of the 5 funds with higher cagr
funds_beta.loc[top5, :].sort_values('beta', ascending = False).head()

### alpha function
Having the funds, ibovespa and selic returns as well as the funds beta we can calculate how much excess of return each fund manager added when compared to the market. Wich funds beat the market and generated more alpha to their clients?  <br>
Techinical reference: investopedia.com/terms/a/alpha.asp

In [ ]:
#joins CAGR and beta dataframes
funds_alpha = funds_cagr.drop(columns = ['fund_quota_cum_return','ibov_price_cum_return','selic_price_cum_return',
                           'days','fund_name']).join(funds_beta[['beta','fund_name']])

#calculates alphas
funds_alpha = alpha(funds_alpha,
                    asset_returns = 'fund_quota_cagr' ,
                    bench_returns = 'ibov_price_cagr',
                    riskfree_returns = 'selic_price_cagr',
                    beta = 'beta')

#top 10 funds by alpha
funds_alpha.sort_values('alpha', ascending = False).head()

### sharpe function
How to evaluate the return of assets compared to their risks? The sharpe ratio is the average return earned in excess of the risk-free rate per unit of volatility (a possible aproximation of risk).  <br>
Techinical reference: investopedia.com/terms/s/sharperatio.asp

In [ ]:
#joins CAGR and vol dataframes
funds_sharpe = funds_cagr.drop(columns = ['fund_quota_cum_return','ibov_price_cum_return','selic_price_cum_return',
                           'days','fund_name']).join(funds_vol[['fund_quota_return_1d_vol', 'fund_name']])

#calculates sharpes
funds_sharpe = sharpe(funds_sharpe,
                      asset_returns = 'fund_quota_cagr',
                      riskfree_returns = 'selic_price_cagr',
                      asset_vol = 'fund_quota_return_1d_vol')

#top 5 funds by sharpe
funds_sharpe.sort_values('sharpe', ascending = False).head()

#### Sharpe in rolling windows

In [ ]:
#calculates the volatility in 1 year rolling windows
funds_vol_yr = volatility(funds_returns,
                          group = 'cnpj_fund',
                          values = ['fund_quota_return_1d'],
                          rolling = True, window_size = 252)

#calculates 1 year rolling windows returns
funds_returns_yr = returns(funds,
                           group = 'cnpj_fund',
                           values = ['fund_quota', 'selic_price'],
                           rolling = True,
                           window_size = 252)

#joins returns and volatilities dataframes
funds_sharpe_yr = funds_returns_yr[['date', 'cnpj_fund', 'fund_name' , 'fund_quota_return_252d','selic_price_return_252d']]\
                  .join(funds_vol_yr[['fund_quota_return_1d_vol_252rw']])

#calculates sharpe ratios
funds_sharpe_yr = sharpe(funds_sharpe_yr,
                         asset_returns = 'fund_quota_return_252d',
                         riskfree_returns = 'selic_price_return_252d',
                         asset_vol = 'fund_quota_return_1d_vol_252rw')

#subsets only funds with the top 5 overall sharpes
top5 = (funds_sharpe.sort_values('sharpe', ascending = False)
         .reset_index()
         .iloc[:5,0] #selects top 5 rows and cnpj column
         .to_list()
        )
funds_sharpe_yr = funds_sharpe_yr[funds_sharpe_yr.cnpj_fund.isin(top5)].dropna(subset = ['sharpe'])



#plots the top 5 funds drawdown over time
fig = px.line(funds_sharpe_yr,
              x='date',
              y='sharpe',
              color = 'fund_name' ,
              title='Top 5 Funds by Sharpe 1 year rolling Window',
              labels={"fund_name": "Fund name", "date":"date"})


fig.update_layout(legend=dict(
    yanchor="bottom",
    y= -0.7,
    xanchor="left",
    x=0.01
))

fig.show()

### sortino function
The Sortino ratio is a variation of the Sharpe ratio that differentiates harmful volatility from total overall volatility by using the asset's standard deviation of negative returns (only downside volatility is considered).  <br>
Techinical reference: investopedia.com/terms/s/sortinoratio.asp

In [ ]:
#calculates volatility of negative returns
funds_neg_returns = funds_returns[funds_returns.fund_quota_return_1d<0] #subsets only negative returns

#calculates the standard deviation of negative returns
funds_neg_vol = volatility(funds_neg_returns, group = 'cnpj_fund', values = ['fund_quota_return_1d'], rolling = False)


#joins CAGR and negative vol dataframes
funds_sortino = funds_cagr.drop(columns = ['fund_quota_cum_return','ibov_price_cum_return','selic_price_cum_return',
                           'days']).join(funds_neg_vol[['fund_quota_return_1d_vol']])

#calculates sortino
funds_sortino = sortino(funds_sortino,
                        asset_returns = 'fund_quota_cagr',
                        riskfree_returns = 'selic_price_cagr',
                        asset_negative_vol = 'fund_quota_return_1d_vol')

#top 5 funds by sortino
funds_sortino.sort_values('sortino', ascending = False).head()

### capture_ratio function
Capture ratios measure the relative performances of an fund in relation to a benchmark in bull an bear markets. Funds with capture ratios < 1 performed well in bear markets relative to the benchmark. Funds with bull capture ratios > 0 performed well in bull market in comparisson to the market. The higher the division (capture_bull/capture_bear) is, the better the funds performance is considered. Negative capture ratios indicate the fund is not correlated with the index (moved in the opposite direction).   <br>
Techinical reference: cleartax.in/s/capture-ratio


In [ ]:
#calculates monthly returns of each fund
funds_mth_returns = returns(funds, group = 'cnpj_fund', values = ['fund_quota', 'ibov_price'],
                            rolling = True, window_size = 21)


#calculates the capture ratios in 21 trading days (1 month) rolling window
funds_capture = capture_ratio(funds_mth_returns,
                              asset_returns = 'fund_quota_return_21d',
                              bench_returns = 'ibov_price_return_21d',
                              returns_frequency = 21,
                              group = 'cnpj_fund')

#adds the fund name to the table
funds_capture = funds_capture.merge(funds.groupby('cnpj_fund')[['fund_name']].first(),
                                    how = 'left',
                                    left_index = True,
                                    right_index = True
                                   )

funds_capture.sort_values('capture_ratio', ascending = False).head()
#We can see that some of the best capture ratios are from funds that invest in US stocks.
#Their benchmark is not the ibovespa index. Therefore, they should be disconsidered from this analysis